In [1]:
# !wget https://raw.githubusercontent.com/gururise/AlpacaDataCleaned/main/alpaca_data_cleaned.json

In [2]:
import openai
import instructor
import json
import os
import malaya
from pydantic import BaseModel, Field
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor, as_completed
from unidecode import unidecode
import re
import random

minimum_len = 15

def simple_cleaning(string):
    return re.sub(r'[ ]+', ' ', unidecode(string).replace('\n', ' ').replace('--', ' ').replace('/', ' ')).strip()

/home/husein/.local/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (5.2.0)/charset_normalizer (2.0.7) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "
/home/husein/dev/malaya/malaya/tokenizer.py:214: FutureWarning: Possible nested set at position 3397
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))
/home/husein/dev/malaya/malaya/tokenizer.py:214: FutureWarning: Possible nested set at position 3927
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))


In [3]:
openai.api_type = "azure"
openai.api_base = "https://husein-ai4-aiservices834896161.openai.azure.com/"
openai.api_version = "2023-07-01-preview"
openai.api_key = ''

engine = 'gpt-35-turbo'

In [4]:
def predict(prompt, max_tokens = 1024):
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt},
       
    ]
    try:
        response = openai.ChatCompletion.create(
            engine=engine,
            messages=messages,
            temperature=1.0,
            max_tokens=max_tokens,
            top_p=1.0
        )
        return response.choices[0]['message']['content']
    except Exception as e:
        print(e)
        return None

In [5]:
with open('alpaca_data_cleaned.json','r') as fopen:
    all_objs = json.load(fopen)

In [6]:
from depth import createConstraintsPrompt, createDeepenPrompt, createConcretizingPrompt, createReasoningPrompt
from breadth import createBreadthPrompt

In [7]:
!mkdir output
# !rm output/*

mkdir: cannot create directory ‘output’: File exists


In [8]:
def generate(instruction, index):
    filename = f'output/{index}.json'
    if os.path.exists(filename):
        return
    
    try:
        evol_prompts = []
        evol_prompts.append(createConstraintsPrompt(instruction))
        evol_prompts.append(createDeepenPrompt(instruction))
        evol_prompts.append(createConcretizingPrompt(instruction))
        evol_prompts.append(createReasoningPrompt(instruction))
        evol_prompts.append(createBreadthPrompt(instruction))

        selected_evol_prompt = random.choice(evol_prompts)
        ins = predict(selected_evol_prompt)
        answer = predict(ins + ', jawab dalam bahasa melayu', max_tokens = 2048)
        d = {"instruction":ins,"output":answer}
        with open(filename, 'w') as fopen:
            json.dump(d, fopen)
    except Exception as e:
        print(e)
        pass

In [9]:
instructions = []
for cur_obj in all_objs:
    instructions.append(cur_obj['instruction'].strip() + '\r\n'+ cur_obj['input'].strip())

In [10]:
from tqdm import tqdm

max_worker = 3
for i in tqdm(range(0, len(instructions), max_worker)):
    b = instructions[i: i + max_worker]
    b = [(ins, i + no) for no, ins in enumerate(b)]
    
    with ThreadPoolExecutor(max_workers=max_worker) as executor:
        futures = {executor.submit(generate, f[0], f[1]): f for f in b}

        for future in as_completed(futures):
            future.result()

  6%|██▏                                   | 994/17254 [00:01<00:26, 618.46it/s]

'content'
unsupported operand type(s) for +: 'NoneType' and 'str'


 15%|█████▌                                | 2539/17254 [04:58<25:03,  9.79it/s]

'content'
unsupported operand type(s) for +: 'NoneType' and 'str'


 16%|█████▉                                | 2716/17254 [06:08<38:10,  6.35it/s]

...

 87%|████████████████████████████████▎    | 15046/17254 [58:59<03:31, 10.42it/s]

'content'
unsupported operand type(s) for +: 'NoneType' and 'str'


 96%|█████████████████████████████████▍ | 16489/17254 [1:04:09<07:18,  1.74it/s]

The response was filtered due to the prompt triggering Azure OpenAI's content management policy. Please modify your prompt and retry. To learn more about our content filtering policies please read our documentation: https://go.microsoft.com/fwlink/?linkid=2198766


100%|███████████████████████████████████| 17254/17254 [1:07:31<00:00,  4.26it/s]
